# **INFERENCE PROCESS FOR NEW IMAGES**


# **1. Connect to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Import folder with images in order to run inference over them**

Import the model file

In [ ]:
################################IMPORTAR EL dataset entero a colab UNA VEZ TIENES LA ESTRUCTURA  ####################################################
####################################################################################


# Copiar el archivo data.yaml desde Drive a la ruta deseada en Colab
!cp -r "/content/drive/MyDrive/images" "/content/dickson_23"
#"/content/dataset/lote_4"

# Verificar que se copió correctamente
!ls -l "/content/dickson_23"

# **3. Install Ultralytics**

In [ ]:
#0. En Colab, primero instala los paquetes y monta tu Drive:  Instalar las librerías necesarias
!pip install ultralytics wandb  # Esto instalará ultralytics y wandb (torch y torchvision generalmente ya vienen instalados en Colab)

import ultralytics
ultralytics.checks()

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 51.3/235.7 GB disk)


# **4. Inference process over the imported folder**

This code divides images in tiles, run the inference, merged the images back and divides them based on confidence prediction

Tiles size and overlap is customizable.

The input directory (folder with images) has to be indicated.

The model directory file has to be imported and indicated.

Class number and names are customizable

IoU and confidence values are customizables.



- The output gives:
1. Folder with all images with annotations
2. Folder with negative images
3. Folder with positive images

  3.1 Subfolder with images positives (confidence > 0.8)

  3.2 Subfolder with images positives (confidence < 0.8 and > 0.5)

  3.3 Subfolder with images positives (confidence < 0.5 )

  
  The presence of one single prrediction > the confidence indicated includes all images inside that subfolder


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import matplotlib.pyplot as plt
import shutil

# ====================
# Parámetros Globales
# ====================
TILE_SIZE = 640           # Tamaño de cada tile en píxeles
OVERLAP = 128             # Solapamiento entre tiles en píxeles
STEP = TILE_SIZE - OVERLAP  # Paso entre tiles

# Rutas (modifica estas rutas según tu estructura)
INPUT_DIR = "/content/dickson_23"              # Directorio con imágenes completas (2025)
OUTPUT_DIR = "/content/dickson_23/resultados"  # Carpeta principal de resultados
# Nota: No se guardarán los tiles en disco para ahorrar espacio.
ANNOTATED_DIR = os.path.join(OUTPUT_DIR, "annotated_full")  # Imágenes completas anotadas

# Carpetas para separar imágenes positivas y negativas
POSITIVAS_SEALS_DIR = os.path.join(OUTPUT_DIR, "positivas", "seals")
NEGATIVAS_DIR = os.path.join(OUTPUT_DIR, "negativas")

# Subcarpetas en positivos (para seals) según el rango de score
SEALS_SCORE_SUBFOLDERS = ["low_score_<0.5", "medium_score_0.5-0.8", "high_score_>0.8"]

def create_directories():
    """Crea todas las carpetas necesarias para el pipeline (solo para la clase seal)."""
    folders = [ANNOTATED_DIR, POSITIVAS_SEALS_DIR, NEGATIVAS_DIR]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)
    for sub in SEALS_SCORE_SUBFOLDERS:
        os.makedirs(os.path.join(POSITIVAS_SEALS_DIR, sub), exist_ok=True)

EXCEL_OUTPUT = os.path.join(OUTPUT_DIR, "detections_summary.xlsx")

# Diccionario de clases: solo se trabaja con "seal"
# Se asume que el modelo devuelve detecciones con clase 1 para seal.
CLASS_NAMES = {0: 'seal'}

# Parámetros para inferencia
INFERENCE_CONF = 0.25
INFERENCE_IOU = 0.5

# Modelo: Ajusta la ruta a tus pesos entrenados
MODEL_PATH = "/content/bestmaxi_80_20_coseno.pt"

# ====================
# Funciones Auxiliares
# ====================

def compute_iou(box1, box2):
    """Calcula el IoU entre dos cajas [x1, y1, x2, y2]."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter_area + 1e-6
    return inter_area / union

def nms_global(detections, iou_threshold=0.5):
    """
    Aplica Non-Maximum Suppression (NMS) a una lista de detecciones.
    Cada detección es una tupla (cls, x1, y1, x2, y2, conf).
    """
    if not detections:
        return []
    detections = np.array(detections)
    final_detections = []
    for cls in np.unique(detections[:, 0]):
        # Solo se trabajará la clase "seal" (se asume que es 1)
        if int(cls) != 0:
            continue
        mask = detections[:, 0] == cls
        d_cls = detections[mask]
        order = d_cls[:, 5].argsort()[::-1]
        d_cls = d_cls[order]
        while len(d_cls) > 0:
            best = d_cls[0]
            final_detections.append(best)
            if len(d_cls) == 1:
                break
            rest = d_cls[1:]
            ious = np.array([compute_iou(best[1:5], box[1:5]) for box in rest])
            keep = np.where(ious < iou_threshold)[0]
            d_cls = rest[keep]
    return final_detections

def get_tiles(image):
    """
    Divide la imagen en tiles, devolviendo una lista de tuplas:
    (tile, (offset_x, offset_y)).
    """
    tiles = []
    h, w = image.shape[:2]
    for y in range(0, h, STEP):
        for x in range(0, w, STEP):
            tile = image[y:min(y + TILE_SIZE, h), x:min(x + TILE_SIZE, w)]
            tile_h, tile_w = tile.shape[:2]
            if tile_h < TILE_SIZE or tile_w < TILE_SIZE:
                padded = np.zeros((TILE_SIZE, TILE_SIZE, 3), dtype=tile.dtype)
                padded[:tile_h, :tile_w] = tile
                tile = padded
            tiles.append((tile, (x, y)))
    return tiles

def process_tile_inference(tile, offset, model):
    """
    Realiza inferencia en un tile y convierte las coordenadas al sistema de la imagen original.
    Devuelve una lista de detecciones en formato (cls, x1, y1, x2, y2, conf).
    Solo se conservan las detecciones de "seal" (clase 1).
    """
    detections = []
    results_tile = model.predict(source=tile, conf=INFERENCE_CONF, iou=INFERENCE_IOU, verbose=False)
    if not results_tile:
        return detections
    pred = results_tile[0]
    boxes = pred.boxes
    if boxes is None or boxes.shape[0] == 0:
        return detections
    preds = boxes.data.cpu().numpy()  # Formato: [x1, y1, x2, y2, conf, cls]
    for row in preds:
        x1_tile, y1_tile, x2_tile, y2_tile, conf, cls = row
        # Solo conservar detecciones de "seal" (clase 1)
        if int(cls) != 0:
            continue
        x1 = x1_tile + offset[0]
        y1 = y1_tile + offset[1]
        x2 = x2_tile + offset[0]
        y2 = y2_tile + offset[1]
        detections.append((int(cls), x1, y1, x2, y2, conf))
    return detections

def process_image(image_path, model):
    """
    Procesa una imagen completa:
      - Divide la imagen en tiles y ejecuta inferencia en cada tile.
      - Reconstruye las detecciones en coordenadas originales.
      - Aplica NMS global.
      - Dibuja las detecciones (bounding boxes y labels) en la imagen anotada.
      - Clasifica la imagen como positiva si se detecta al menos una caja (seal).
      - Guarda la imagen anotada (con las etiquetas) en la carpeta de positivos de seals,
        utilizando subcarpetas de acuerdo a un rango de score.
      - Retorna un resumen, la imagen anotada y la lista de detecciones filtradas.
    """
    image = cv2.imread(image_path)
    if image is None:
        return None, None, []
    orig_h, orig_w = image.shape[:2]
    annotated_img = image.copy()
    all_detections = []

    # Obtener tiles (no se guardan en disco)
    tiles = get_tiles(image)
    for tile, offset in tiles:
        detections = process_tile_inference(tile, offset, model)
        all_detections.extend(detections)

    # Aplicar NMS global
    detections_filtered = nms_global(all_detections, iou_threshold=0.5)

    # Dibujar las detecciones en la imagen anotada
    for det in detections_filtered:
        cls, x1, y1, x2, y2, conf = det
        cv2.rectangle(annotated_img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # rojo
        label = f"{CLASS_NAMES.get(cls, str(cls))}:{conf:.2f}"
        cv2.putText(annotated_img, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)  # rojo

    # Clasificación: se considera la imagen positiva si hay detecciones
    if len(detections_filtered) == 0:
        pos_flag = 0
        neg_flag = 1
    else:
        pos_flag = 1
        neg_flag = 0

    # Para imágenes positivas, calculamos el promedio de score y guardamos la imagen anotada
   # Para imágenes positivas, clasificamos según la máxima confianza
    if pos_flag:
        max_score = np.max([d[5] for d in detections_filtered])

        if max_score < 0.5:
            score_folder = "low_score_<0.5"
        elif max_score <= 0.8:
            score_folder = "medium_score_0.5-0.8"
        else:
            score_folder = "high_score_>0.8"

        # Ruta de carpeta anotada
        annotated_target_dir = os.path.join(POSITIVAS_SEALS_DIR, score_folder)
        os.makedirs(annotated_target_dir, exist_ok=True)
        cv2.imwrite(os.path.join(annotated_target_dir, os.path.basename(image_path)), annotated_img)

        # Ruta de carpeta con originales (sin labels)
        originals_target_dir = os.path.join(POSITIVAS_SEALS_DIR, score_folder + "_originales")
        os.makedirs(originals_target_dir, exist_ok=True)
        shutil.copy2(image_path, os.path.join(originals_target_dir, os.path.basename(image_path)))

        # También guardamos en annotated_full (como antes)
        cv2.imwrite(os.path.join(ANNOTATED_DIR, os.path.basename(image_path)), annotated_img)

    else:
        shutil.copy2(image_path, os.path.join(NEGATIVAS_DIR, os.path.basename(image_path)))

    # Resumen: sólo se cuenta la detección de "seal"
    rec = {"imagen": os.path.basename(image_path), "positive": pos_flag, "negative": neg_flag, "seal": len(detections_filtered)}
    rec["total_detecciones"] = len(detections_filtered)

    return rec, annotated_img, detections_filtered

def process_all_images(model):
    """Procesa todas las imágenes en INPUT_DIR y acumula resúmenes y detecciones."""
    all_summary = []
    global_detections = []
    num_images = 0
    for img_file in os.listdir(INPUT_DIR):
        if not img_file.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        num_images += 1
        img_path = os.path.join(INPUT_DIR, img_file)
        rec, annotated_img, detections_filtered = process_image(img_path, model)
        if rec is not None:
            all_summary.append(rec)
            global_detections.extend(detections_filtered)
            cv2.imwrite(os.path.join(ANNOTATED_DIR, img_file), annotated_img)
    return all_summary, global_detections, num_images

def compute_global_statistics(global_detections, num_images):
    """Calcula estadísticas globales a partir de las detecciones acumuladas."""
    stats = {}
    total_det = len(global_detections)
    stats["total_detecciones"] = total_det
    stats["promedio_detecciones_por_imagen"] = total_det / num_images if num_images > 0 else 0
    if total_det > 0:
        confs = [d[5] for d in global_detections]
        stats["conf_mean"] = np.mean(confs)
        stats["conf_median"] = np.median(confs)
        stats["conf_std"] = np.std(confs)
    else:
        stats["conf_mean"] = stats["conf_median"] = stats["conf_std"] = 0
    return stats

def compute_class_statistics(global_detections):
    """Calcula estadísticas para la clase 'seal'."""
    class_stats = {}
    dets = [d for d in global_detections if d[0] == 0]
    if dets:
        confs = [d[5] for d in dets]
        areas = [(d[3]-d[1])*(d[4]-d[2]) for d in dets]
        class_stats["seal"] = {
            "num_detecciones": len(dets),
            "conf_mean": np.mean(confs),
            "conf_median": np.median(confs),
            "conf_std": np.std(confs),
            "area_mean": np.mean(areas),
            "area_median": np.median(areas),
            "area_std": np.std(areas)
        }
    else:
        class_stats["seal"] = {
            "num_detecciones": 0,
            "conf_mean": 0,
            "conf_median": 0,
            "conf_std": 0,
            "area_mean": 0,
            "area_median": 0,
            "area_std": 0
        }
    return class_stats

def save_excel_summary(summary_records, global_stats, class_stats):
    """
    Exporta tres hojas a un único archivo Excel:
      - "Resumen por Imagen": una fila por imagen + una fila TOTAL.
      - "Estadísticas Globales": estadísticas globales.
      - "Estadísticas por Clase": estadísticas detalladas para la clase 'seal'.
    Se utiliza pd.concat para agregar la fila TOTAL.
    """
    df_images = pd.DataFrame(summary_records)
    totals = {"imagen": "TOTAL"}
    for col in df_images.columns:
        if col != "imagen":
            if pd.api.types.is_numeric_dtype(df_images[col]):
                totals[col] = df_images[col].sum()
            else:
                totals[col] = ""
    df_images = pd.concat([df_images, pd.DataFrame([totals])], ignore_index=True)

    df_global = pd.DataFrame([global_stats])

    rows = []
    for cls_name, stats in class_stats.items():
        row = {"clase": cls_name}
        row.update(stats)
        rows.append(row)
    df_class = pd.DataFrame(rows)

    with pd.ExcelWriter(EXCEL_OUTPUT) as writer:
        df_images.to_excel(writer, sheet_name="Resumen por Imagen", index=False)
        df_global.to_excel(writer, sheet_name="Estadísticas Globales", index=False)
        df_class.to_excel(writer, sheet_name="Estadísticas por Clase", index=False)
    print("Resumen y estadísticas guardadas en:", EXCEL_OUTPUT)

def run_pipeline():
    create_directories()
    model = YOLO(MODEL_PATH)
    summary_records, global_detections, num_images = process_all_images(model)
    global_stats = compute_global_statistics(global_detections, num_images)
    class_stats = compute_class_statistics(global_detections)
    save_excel_summary(summary_records, global_stats, class_stats)

# Ejecutar el pipeline completo
run_pipeline()


Resumen y estadísticas guardadas en: /content/dickson_23/resultados/detections_summary.xlsx


# **5. Save the results into a folder in Google Drive**

In [ ]:
# 10. Guardar los resultados del experimento

# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/Dickson_23"
!mkdir -p "{dest_folder}"  # El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r "/content/dickson_23/resultados" "{dest_folder}/"  # Copiar los resultados del modelo en drive

# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"



resultados
